In [4]:
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models,layers


In [ ]:

train_dir = "seg_train"
test_dir = "seg_test"
pred_dir = "seg_pred"

img_size = (150, 150)  
batch_size = 32        


In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,        
    rotation_range=20,     
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range=0.2,        
    shear_range=0.2,       
    horizontal_flip=True   
)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

In [ ]:
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

In [ ]:
import os


train_dir = "seg_train"
test_dir = "seg_test"

def count_images(directory):
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            num_images = len(os.listdir(class_path))
            print(f"{directory}/{class_name}: {num_images} images")

print("Training Set:")
count_images(train_dir)

print("Test Set:")
count_images(test_dir)

In [ ]:
model = models.Sequential()


model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))


model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(train_gen.num_classes, activation="softmax"))


In [ ]:
model.compile(
    optimizer="adam", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10
)


In [ ]:
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc:.2f}")


In [ ]:
pred_gen = test_datagen.flow_from_directory(
    pred_dir,
    target_size=img_size,
    batch_size=1,
    class_mode=None,
    shuffle=False)

predictions = model.predict(pred_gen)
predicted_classes = predictions.argmax(axis=-1)


labels = list(train_gen.class_indices.keys())
predicted_labels = [labels[i] for i in predicted_classes]

print("Sample Predictions:", predicted_labels[:10])


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


test_gen.reset()  
Y_pred = model.predict(test_gen, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

y_true = test_gen.classes


class_labels = list(test_gen.class_indices.keys())


cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_labels, yticklabels=class_labels, cmap="Blues")
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


report = classification_report(y_true, y_pred, target_names=class_labels)
print("Classification Report:\n", report)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


for i in range(3,9):
    img= next(pred_gen)  
    plt.imshow(img[0])
    plt.axis("off")
    plt.show()

    pred = model.predict(img)
    pred_class = labels[np.argmax(pred)]
    print("Predicted:", pred_class)


In [ ]:
import numpy as np
from collections import Counter

counts = Counter(predicted_labels)
print("Prediction Counts:")
for label, count in counts.items():
    print(f"{label}: {count}")
